# Part 1: Data Owner Domain Setup

In this notebook, you will walk through the steps to setup a single-machine Domain node, load some data into it, and create two user accounts:

- *Data Compliance Officer:* someone who has the ability to respond to data-access requests
- *Data Scientist:* someone who will find your data, run an experiment, and then create a data-access request to download their results.

### Step 1: Clone PyGrid
First begin by cloning PyGrid (https://github.com/OpenMined/PyGrid) and cd-ing into the Domain application directory.


```bash
git clone https://github.com/OpenMined/PyGrid.git
cd apps/domain
```

### Step 2: Install Poetry (if you don't have it)

Now we need to ensure that poetry is installed. The following command should work for you (OSX/linux/bashonwindows), but if it doesn't visit the poetry docs (https://python-poetry.org/docs/)

```bash
curl -sSL https://raw.githubusercontent.com/python-poetry/poetry/master/get-poetry.py | python -
```

### Step 3: Install Poetry Dependencies

If you're following this tutorial you should still be in the "PyGrid/apps/domain" folder of PyGrid. From that folder, run the following command:

```bash
poetry install
```

### Step 4 (Optional): Reset Node first?
If you've run a single-machine PyGrid domain before on this machine, you might have a "nodedatabase.db" file in "PyGrid/Apps/domain/src/nodedatabase.db". If you'd like to refresh your machine from scratch run the following (again assuming your CWD is "PyGrid/apps/domain".

```bash
rm src/nodedatabase.db
```

### Step 5: Launch Domain Node

Now with all your dependencies installed, you're ready to launch the domain node!

```bash
./run.sh --port 5000 --start_local_db
```

### Step 6: Create Root Owner and Login

The node is currently launched in "sleep mode" (you can see by visiting 'http://localhost:5000/'). This means that there isn't a root user yet and you need to declare one.

In [1]:
from pathlib import Path
import os
cwd = Path().resolve()

In [2]:
grid_relative = f"{cwd}/../../../../../grid"
grid_path = Path(os.path.abspath(grid_relative))
db_path = grid_path / "apps/domain/src/nodedatabase.db"
print(db_path)

/Users/madhavajay/dev/PySyft/packages/grid/apps/domain/src/nodedatabase.db


In [3]:
import sqlite3
cursor = sqlite3.connect(db_path).cursor()

def q(msg):
    cursor.execute(msg)
    return cursor.fetchall()

def db():
    for table in q("SELECT name FROM sqlite_master WHERE type='table';"):
        if table[0] != "group":
            res = q("Select * from " + str(table[0]) + ";")
            if len(res) > 0:
                print()
                print(table[0])                
                print(res)

In [4]:
import syft as sy
import torch as th
import numpy as np

# UNCOMMENT THIS FOR FIRST-TIME SETUP
try:
    sy.login().setup(email="owner@myorg.com", password="owerpwd", domain_name="Trask's Node")
except Exception as e:
    print("This PyGrid domain already has an owner!")

# LOGIN AS ROOT USER
root_domain = sy.login(email="owner@myorg.com", password="owerpwd")

This PyGrid domain already has an owner!


In [5]:
root_domain.store

,ID,Tags,Description,object_type
0,<UID: 1a8c13a7f11e43ff930c531acfa7a6d5>,[a],,<class 'syft.lib.python.Int'>
1,<UID: 244c19aadaec46bbbd047d3b9d20719d>,[data],,<class 'torch.Tensor'>
2,<UID: 33b15dbac58b4058ab31ac43e8431a54>,[model],,<class 'torch.nn.modules.linear.Linear'>
3,<UID: 608d3745f8ec4c348652541391155c84>,[],,<class 'torch.Tensor'>
4,<UID: 64914e74cf81412d93f818d7e6f7004f>,[y],,<class 'torch.Tensor'>
5,<UID: 70ea239981494b84a2561cd02146c7e9>,"[#xor, #inputs, #data_01.csv]",,<class 'torch.Tensor'>
6,<UID: ee3fd0727b0a4f1dbdcb5019edb61e16>,"[#xor, #targets, #data_01.csv]",,<class 'torch.Tensor'>


In [6]:
a = sy.lib.python.Int(1)
a_ptr = a.send(root_domain, tags=["a"])

In [7]:
a_ptr.id_at_location

<UID: 46fac3351be94a75842a54d80faf77fb>

In [8]:
a_ptr.get(delete_obj=False)

1

In [9]:
db()


bin_object
[('1a8c13a7-f11e-43ff-930c-531acfa7a6d5', b'\n\x13syft.lib.python.Int\x12\x16\x08\x01\x12\x12\n\x10\xee1Aky\xa6C\xa1\xb6\xf4\x9b\x13\xc8\x19\xacS', 'Int'), ('33b15dba-c58b-4058-ab31-ac43e8431a54', b'\n$syft.wrappers.torch.nn.ModuleWrapper\x12\x97\x06\n\x06Linear\x1a(in_features=2, out_features=1, bias=True2\xe2\x05\n6\n\x16syft.lib.python.String\x12\x1c\n\x06weight\x12\x12\n\x10\xb6:O\xeb\xf3%N\xbf\x99$:\xd2\x18\x1fZA\n4\n\x16syft.lib.python.String\x12\x1a\n\x04bias\x12\x12\n\x10\xf2H[#\x1d\x9dNu\xac\x02~?c\x83\xf0\xff\x12\xcf\x02\n1syft.wrappers.torch.nn.parameter.ParameterWrapper\x12\x99\x02\n\x94\x02*\x88\x02\xff\xff\xff\xff\xf8\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x1a\x00\x06\x00\x05\x00\x08\x00\x0c\x00\x0c\x00\x00\x00\x00\x04\x04\x00 \x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0e\x00(\x00\x07\x00\x08\x00\x0c\x00\x10\x00\x14\x00\x0e\x00\x00\x00\x00\x00\x00\x03\x8c\x00\x00\x004\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [10]:
try:
    # if model exists skip
    root_domain.store["model"]
except Exception as e:
    if "More than one" not in str(e):
        # model doesnt exist so make it
        model = th.nn.Linear(2,1)
        model_ptr = model.send(root_domain, tags=["model"])

In [11]:
root_domain.store

,ID,Tags,Description,object_type
0,<UID: 1a8c13a7f11e43ff930c531acfa7a6d5>,[a],,<class 'syft.lib.python.Int'>
1,<UID: 244c19aadaec46bbbd047d3b9d20719d>,[data],,<class 'torch.Tensor'>
2,<UID: 33b15dbac58b4058ab31ac43e8431a54>,[model],,<class 'torch.nn.modules.linear.Linear'>
3,<UID: 46fac3351be94a75842a54d80faf77fb>,[a],,<class 'syft.lib.python.Int'>
4,<UID: 608d3745f8ec4c348652541391155c84>,[],,<class 'torch.Tensor'>
5,<UID: 64914e74cf81412d93f818d7e6f7004f>,[y],,<class 'torch.Tensor'>
6,<UID: 70ea239981494b84a2561cd02146c7e9>,"[#xor, #inputs, #data_01.csv]",,<class 'torch.Tensor'>
7,<UID: ee3fd0727b0a4f1dbdcb5019edb61e16>,"[#xor, #targets, #data_01.csv]",,<class 'torch.Tensor'>


In [12]:
try:
    root_domain.users.create(
        email="compliance@myorg.com", password="pwd123", role="Compliance Officer"
    )
except Exception as e:
    print("Compliance Officer already created.")

[2021-06-15T16:25:10.478615+1000][CRITICAL][logger]][77687] UnknownPrivateException has been triggered.


Compliance Officer already created.


In [13]:
root_domain.store

,ID,Tags,Description,object_type
0,<UID: 1a8c13a7f11e43ff930c531acfa7a6d5>,[a],,<class 'syft.lib.python.Int'>
1,<UID: 244c19aadaec46bbbd047d3b9d20719d>,[data],,<class 'torch.Tensor'>
2,<UID: 33b15dbac58b4058ab31ac43e8431a54>,[model],,<class 'torch.nn.modules.linear.Linear'>
3,<UID: 46fac3351be94a75842a54d80faf77fb>,[a],,<class 'syft.lib.python.Int'>
4,<UID: 608d3745f8ec4c348652541391155c84>,[],,<class 'torch.Tensor'>
5,<UID: 64914e74cf81412d93f818d7e6f7004f>,[y],,<class 'torch.Tensor'>
6,<UID: 70ea239981494b84a2561cd02146c7e9>,"[#xor, #inputs, #data_01.csv]",,<class 'torch.Tensor'>
7,<UID: ee3fd0727b0a4f1dbdcb5019edb61e16>,"[#xor, #targets, #data_01.csv]",,<class 'torch.Tensor'>


In [14]:
# db()

In [15]:
# model_ptr.get()

In [16]:
# db()

In [17]:
# root_domain.store

In [18]:
# root_domain.datasets

# Create and Upload a Few Datasets

We can add datasets to the Domain using our root-user login. Note that unless you've done something special no other user is able to add datasets by default.

In [19]:
x = th.tensor([1,2,3,4]).send(root_domain)

In [20]:
# db()

In [21]:
xor_inputs = np.array([[0,0],
                       [0,1],
                       [1,0],
                       [1,1]])

xor_input_dataset = root_domain.upload_dataset(data=xor_inputs, 
                                          description="XOR dataset inputs", 
                                          tags=['#xor', '#inputs'])

xor_targets = np.array([[0],
                       [1],
                       [1],
                       [0]])

xor_target_dataset = root_domain.upload_dataset(data=xor_targets, 
                                          description="XOR dataset targets", 
                                          tags=['#xor', '#targets'])

In [22]:
root_domain.datasets

,id,manifest,description,tags,data
0,c726fe8d-c7ed-494c-99c5-e613109776fd,XOR dataset inputs,XOR dataset inputs,"[#xor, #inputs]","[{'name': 'tmp/data_01.csv', 'id': '70ea2399-8..."
1,06d35eeb-0cae-4c8e-9bca-8437b1ad8350,XOR dataset targets,XOR dataset targets,"[#xor, #targets]","[{'name': 'tmp/data_01.csv', 'id': 'ee3fd072-7..."
2,ac124d65-306e-49cf-803f-202103a7f8f1,XOR dataset inputs,XOR dataset inputs,"[#xor, #inputs]","[{'name': 'tmp/data_01.csv', 'id': 'b64665ec-5..."
3,9b533c42-fa15-4e85-a3ba-a064fc9bd162,XOR dataset targets,XOR dataset targets,"[#xor, #targets]","[{'name': 'tmp/data_01.csv', 'id': '6dea71a8-6..."


# Create a Data Compliance Officer

Eventually a Data Scientist is going to need some management, particularly when they want to download their statistical results. While this doesn't always need manual review, sometimes it does and it's important we create a user who is capable of reviewing and approving data access requests.

In [23]:
root_domain.roles

,id,name,can_triage_requests,can_edit_settings,can_create_users,can_create_groups,can_edit_roles,can_manage_infrastructure,can_upload_data
0,1,User,False,False,False,False,False,False,False
1,2,Compliance Officer,True,False,False,False,False,False,False
2,3,Administrator,True,True,True,True,False,False,True
3,4,Owner,True,True,True,True,True,True,True


In [24]:
root_domain.users

,id,email,private_key,verify_key,role,groups
0,1,owner@myorg.com,9c7700cd631502f0a071be9028b21bb73dba1ab838a2db...,a0dae1eba2ac6469f2992e44412a90661131ef575155a9...,4,[]
1,2,compliance@myorg.com,e33ea8ea9c367acf7c92c908cfeb25cb5b9e8b2c22e18d...,2c38ad2413731c24c565108ee83b96784a8d269fbdbee9...,2,[]
2,3,scientist6@researchorg.edu,63f62256c8df3f7abc3ff04e9a7032083983dc2d2a5fa5...,915520156fdd566da28fda6ef2f1c8208009695de63fa7...,1,[]


# Done with Part 1!

You may now proceed to Part 2!

In [25]:
root_domain.store

,ID,Tags,Description,object_type
0,<UID: 1a8c13a7f11e43ff930c531acfa7a6d5>,[a],,<class 'syft.lib.python.Int'>
1,<UID: 244c19aadaec46bbbd047d3b9d20719d>,[data],,<class 'torch.Tensor'>
2,<UID: 33b15dbac58b4058ab31ac43e8431a54>,[model],,<class 'torch.nn.modules.linear.Linear'>
3,<UID: 46fac3351be94a75842a54d80faf77fb>,[a],,<class 'syft.lib.python.Int'>
4,<UID: 608d3745f8ec4c348652541391155c84>,[],,<class 'torch.Tensor'>
5,<UID: 64914e74cf81412d93f818d7e6f7004f>,[y],,<class 'torch.Tensor'>
6,<UID: 6dea71a863e4405ebba2bf8976d53de1>,"[#xor, #targets, #data_01.csv]",,<class 'torch.Tensor'>
7,<UID: 70ea239981494b84a2561cd02146c7e9>,"[#xor, #inputs, #data_01.csv]",,<class 'torch.Tensor'>
8,<UID: 7214fab037fa469794f491252d9c763c>,[],,<class 'torch.Tensor'>
9,<UID: b64665ec54904aa39f19250318fc9957>,"[#xor, #inputs, #data_01.csv]",,<class 'torch.Tensor'>


In [26]:
db()


bin_object
[('1a8c13a7-f11e-43ff-930c-531acfa7a6d5', b'\n\x13syft.lib.python.Int\x12\x16\x08\x01\x12\x12\n\x10\xee1Aky\xa6C\xa1\xb6\xf4\x9b\x13\xc8\x19\xacS', 'Int'), ('33b15dba-c58b-4058-ab31-ac43e8431a54', b'\n$syft.wrappers.torch.nn.ModuleWrapper\x12\x97\x06\n\x06Linear\x1a(in_features=2, out_features=1, bias=True2\xe2\x05\n6\n\x16syft.lib.python.String\x12\x1c\n\x06weight\x12\x12\n\x10\xb6:O\xeb\xf3%N\xbf\x99$:\xd2\x18\x1fZA\n4\n\x16syft.lib.python.String\x12\x1a\n\x04bias\x12\x12\n\x10\xf2H[#\x1d\x9dNu\xac\x02~?c\x83\xf0\xff\x12\xcf\x02\n1syft.wrappers.torch.nn.parameter.ParameterWrapper\x12\x99\x02\n\x94\x02*\x88\x02\xff\xff\xff\xff\xf8\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x1a\x00\x06\x00\x05\x00\x08\x00\x0c\x00\x0c\x00\x00\x00\x00\x04\x04\x00 \x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0e\x00(\x00\x07\x00\x08\x00\x0c\x00\x10\x00\x14\x00\x0e\x00\x00\x00\x00\x00\x00\x03\x8c\x00\x00\x004\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x